<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Classification with K-Nearest Neighbors

_Authors: Kiefer Katovich (SF), Alexander Barriga (SF), Joseph Nelson (DC)_

---

### Learning Objectives
- Understand the difference between classification and regression models
- Understand the K-Nearest Neighbors algorithm visually and in pseudocode
- Explain the differences between distance metrics and explore the two most common
- Apply KNN classification to the Wisconsin breast cancer dataset
- Practice manually performing stratified cross-validation
- Visually examine the effect of K neighbors on the decision boundary
- Explain the effect of choosing K on the bias-variance tradeoff

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='intro'></a>

## Introduction: regression vs. classification

We've discussed the difference between continuous and discrete numbers. That difference is what distinguishes classification from regression prediction tasks. Today we are going to focus on predicting non-quantitative, discrete categories, which is known as classification.

Take wine for example. You could predict a quality rating using regression, but what if we just wanted to predict whether wine was good or bad? Red or white? 

Classification algorithms do just that; they predict categories, or classes. Split the data into groups and place new data into those groups. 

![](http://ipython-books.github.io/images/ml.png "Best Split vs Best Fit")

<a id='knn-visual-intro'></a>

---

### K-Nearest Neighbors (KNN) visually

**KNN works similarly to how we humans might choose to classify things. Below we have some red and blue dots:**
![Alt text](http://blog.yhat.com/static/img/knn_reds_and_blues.png "Some Dots")

**A new dot appears without a color and we need to decide which color it is most likely going to be.**
![Alt text](http://blog.yhat.com/static/img/knn_new_point.png "A New Dot Appears")

**We compare it to its three nearest neighbors – its neighbors are more often red, so we label it red.**
![Alt text](http://blog.yhat.com/static/img/knn_new_point_pred.png "3 Nearest Neighbors")

**What if we increase the number of neighbors to consider to 5?**
![Alt text](http://blog.yhat.com/static/img/knn_new_point_pred_blue.png "5 Nearest Neighbors")

**This is in essence the K-Nearest Neighbors (KNN) algorithm. The K represents the number of "neighbors" you use.**

> ***Images above credited to the yhat blog.***

<a id='knn'></a>

## The KNN algorithm

---

K-Nearest Neighbors takes a unique approach to finding patterns in the data. In order to estimate a value (regression) or class membership (classification), the algorithm finds the observations in its training data that are "nearest" to the observation to predict. It then takes a vote of those training observations' target values to estimate the value for the new data point.

Distance is usually calculated using the euclidean distance. The "K" in KNN refers to the number of nearest neighbors that will be contributing to the prediction. 

Today we will be looking at KNN only in the context of classification.

**The KNN can be concisely represented with pseudocode:**

```
for unclassified_point in sample:
    for known_point in known_class_points:
        calculate distances (euclidean or other) between known_point and unclassified_point
    for k in range of specified_neighbors_number:
        find k_nearest_points in known_class_points to unclassified_point
    assign class to unclassified_point using "votes" from k_nearest_points
```

> **Note**: in the case of ties, sklearn's `KNeighborsClassifier()` will just choose the first class (when weights are uniform)! If this is unappealing to you you can change the weights keyword argument to 'distance'. More on this later.



<a id='distance'></a>
## The KNN distance metric

---
KNN typically uses one of two distance metrics: euclidean or manhattan. Other distance metrics are possible, but more rare (sometimes it makes sense to create your own distance function.

<a id='euclidean'></a>
### Euclidean distance

Recall the famous Pythagorean Theorem
![Alt text](http://ncalculators.com/images/pythagoras-theorem.gif)

We can apply the theorem to calculate distance between points. This is called Euclidean distance. 

![Alt text](http://rosalind.info/media/Euclidean_distance.png)

### $$\text{Euclidean  distance}=\sqrt{(x_1-x_2)^2+(y_1-y_1)^2}$$

There are many different distance metrics, but Euclidean is the most common (and default in sklearn).


<a id='wisconsin'></a>

## Load the wine dataset

---

Below we will be testing out the KNN classification algorithm on the classic [UCI Wine Dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/)

In [61]:
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier 
import pandas as pd
import numpy as np
import seaborn as sns

In [21]:
#we are looking to classify wine 
wine_loader = load_wine()

wine_loader
#not as nice as a CSV, which we would typically open
#we are going to discuss how to troubleshoot 
#for this there is a {} brackets so it might be a dictonary, there is also a Key 'DESCR'
#and the colon 
type(wine_loader)
#its inherited from a dicetionary 
isinstance(wine_loader, dict)
#tells us if it is a dictionary OR built on top of a dictionary
#what does built on top of a dictionary mean??? 

True

In [22]:
for key, val in wine_loader.items():
    print key

target_names
data
target
DESCR
feature_names


In [23]:
print wine_loader["DESCR"]

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

---

### <font color=blue>Independent - </font>Create a DataFrame from this strange `wine` object. Don't forget to add column names.


In [25]:
# A:
# DataFrame.from_dict
# DataFrame.wine_loader
type(wine_loader)


sklearn.utils.Bunch

In [37]:
# data1 = pd.DataFrame(data= np.c_[iris['data'],
df = pd.DataFrame([wine_loader])
df.head()
df.columns
type(df)
#changed the wine_loader into a dataframe to be able to better 
#manipulate the data

pandas.core.frame.DataFrame

In [115]:
#Gus's way
wine_loader.data
#this is a numpy array and you can use this, but it wil make things
#difficult down the line because there aren't any column names
y = wine_loader.target
wine_df = pd.DataFrame(wine_loader.data, columns=wine_loader.feature_names)

In [57]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

<a id='correlations'></a>
## Examine the correlation structure of the dataset

---

You should exclude the `id` column as this is just an indicator variable for the subject.

<a id='heatmap'></a>
### Method 1: plot a heatmap of the correlation matrix

Plot a seaborn heatmap of the correlation matrix to visually examine which variables are correlated and anti-correlated, and to what degree.

In [70]:
# wine_df.corr()
sns.heatmap(wine_df.corr())

In [69]:
# A:
sns.pairplot(wine_df.iloc[:, :5])

<a id='pairplot'></a>
### Method 2: Use seaborn's pairplot to visualize relationships between variables

When you have a small number of predictor variables, seaborn's `pairplot` function will give you a more detailed visual look at the relationships between variables. The pairplot is similar to a correlation matrix, but displays scatterplots of variable pairs. Along the diagonal line are histograms showing the distribution of each variable.

One of the most appealing aspects of the pairplot function for classification tasks is that the scatterplots and histograms can be split along a hue variable. If we use the `malignant` target class as the hue we are able to see how the classes are distributed across these variables as well.

Plot data using seaborn's `pairplot()` function. The hue will be the class variable "malignant". The variables will be the other columns excluding, of course, the subject ID column. This function can take some time to run.

> **Note:** Most of these predictors are highly correlated with the "class" variable. This is already an indication that our classifier is very likely to perform well.

In [84]:
# A:

#1. import 
from sklearn.neighbors import KNeighborsClassifier

#2. instantiate ---- the paranthesis is critical to differntiating
# so that it instantiate
knn = KNeighborsClassifier(n_neighbors=5)
#5 is ?????

#3. fit (aka "train" on your labeled data)

knn.fit(wine_df, y)
#a method is a function that requires parathesis (fit is a method)

knn.predict(wine_df.sample(3))
#gives array with the class type predictions
#the numerical lables 

# 4.Make a new prediction or score
wine_sample = wine_df.sample(5)

print "prediction:", knn.predict(wine_sample)
print "actual: ", y[wine_sample.index]


 prediction: [2 2 2 1 0]
actual:  [2 2 1 1 0]


<a id='kneighborsclassifier'></a>

## Using sklearn's `KNeighborsClassifier`

---

Let's see how the sklearn KNN classifier performs on our dataset predicting the malignant target class using cross-validation.

Load the KNN classifier like so:
```python
from sklearn.neighbors import KNeighborsClassifier
```

**We are going to set some arguments when instantiating the model:**
1. **n_neighbors** specifies how many neighbors will vote on the class
2. **weights** uniform weights indicate that all neighbors have the same weight
3. **metric** and **p**: when distance is minkowski (the default) and p == 2 (the default), _this is equivalent to the euclidean distance metric_


In [9]:
# A:

<a id='target-predictors'></a>
### Create your target vector and predictor matrix

The target should be the binary `malignant` column. The predictors are up to you.

In [10]:
# A: See above 

### Fit our first K-nearest neighbors model to the wine data

The steps to using an sklearn model are:
1. Instantiated the model
2. Fit the instantiated model object to day (`X` and `y`)
3. Score or make predictions with your "trained model"

<a id='baseline'></a>
### Calculate the "baseline" accuracy

Before we can evaluate whether our classifier's accuracy is good or bad, we need to know the baseline accuracy.

**The baseline accuracy is the proportion of the majority class.**

For a binary classification, this means that the baseline accuracy is the percent of the dataset that is labeled malignant or benign, depending on whichever of malignant or benign is greater. This can be calculated:

```python
baseline = np.mean(y)  # if np.mean(y) is >= 0.5
baseline = 1. - np.mean(y) # if np.mean(y) is < 0.5
```

**It is critical that you know your baseline accuracy!**

If your dataset for example had 95 1's and 5 0's, and you got a 95% accuracy using KNN, if you had not looked at your baseline accuracy you may conclude that your classifier is doing great. In fact, it's doing no better than chance! The classifier could have guessed only 1's and gotten a 95% accuracy.

In [102]:
# A:
pd.Series(y).value_counts() / len(y)

1    0.398876
0    0.331461
2    0.269663
dtype: float64

<a id='cv-knn5'></a>
### What is the accuracy for a KNN model with 5 neighbors?

In [103]:
# A:
y_hat = knn.predict(wine_df)
(y_hat == y).mean()

0.7865168539325843

In [107]:
#equivalent to above
knn.score(wine_df,y)

#when considering a model two key factors:

0.7865168539325843

<a id='cv-knn1'></a>
## Tuning your model for performance

As you can see the accuracy is already very high with 5 neighbors on the full dataset, but in industry, mere percentage point gains in performance could be a matter of millions of dollars. Let's see how well we can do!

Right now we have two main dials to turn:
    1. Feature selection ie which column or columns to include in the training set
    2. Choice of `n_neighbors` aka `k`

### <font color=blue>Partner Work (25 minutes)</font>   - Feature selection
We are going to start on feature selection. With your partner, start by exploring which columns will produce the best accuracy score using the default `n_neighbors=5`.

After our exploration, we will convene to share our findings as a class.

#### Teams

<img src=https://i.imgur.com/HFFCeUH.png width="30%" align=left>

In [ ]:
#Feature selection strategies:
# -segmenting based on the different class labels (is the mean different but also considering
#                                                 variance)
#     - group bys
#     - overlaid histograms
-trial and error 
    - check features (comparing/combined columns)
    - brute force





In [112]:
df.feature_names

0    [alcohol, malic_acid, ash, alcalinity_of_ash, ...
Name: feature_names, dtype: object

In [114]:
#1. import 
from sklearn.neighbors import KNeighborsClassifier

#2. instantiate 
knn = KNeighborsClassifier(n_neighbors=100)

#3. fit (aka "train" on your labeled data)
knn.fit(wine_df, y)
knn.predict(wine_df.sample(100))

# 4.Make a new prediction or score
wine_sample = wine_df.sample(100)

print "prediction:", knn.predict(wine_sample)
print "actual: ", y[wine_sample.index]

prediction: [2 0 1 1 1 1 2 2 1 1 2 1 1 1 1 1 1 2 2 0 2 2 1 1 0 0 1 1 0 1 0 1 1 1 0 1 0
 2 1 0 1 1 1 1 1 1 0 2 1 2 1 1 0 1 2 0 1 0 1 2 0 0 1 0 1 2 1 0 2 2 1 1 1 2
 0 1 1 2 1 1 1 1 2 0 0 1 2 0 1 0 0 1 2 1 2 0 1 1 1 2]
actual:  [2 0 1 1 1 1 2 2 1 1 2 2 2 2 1 1 2 1 1 0 0 0 2 1 0 0 2 2 0 1 0 2 1 1 0 1 0
 1 2 0 2 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 2 2 0 0 1 0 1 0 2 0 1 2 1 2 2 2
 0 2 1 0 1 1 2 1 1 0 0 1 1 0 1 0 1 1 0 1 2 0 2 1 1 1]


In [109]:
# A:
pd.Series(y).value_counts() / len(y)

1    0.398876
0    0.331461
2    0.269663
dtype: float64

In [110]:
y_hat = knn.predict(wine_df)
(y_hat == y).mean()

0.6966292134831461

### <font color=blue>Partner Work (25 minutes)</font>   - Hyper-parameter tuning

Using the best features, explore what the best value of `k` is for this dataset. 

In [117]:
# A: 

for k in range(1,30)[::-1]:
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(wine_df,y)
    print(k, knn.score(wine_df,y))
    
#using the training set we got 100% b/c the points are being scored agansit itself 
#the points that weren't used in training are key to test out 
    

(29, 0.7247191011235955)
(28, 0.7247191011235955)
(27, 0.7247191011235955)
(26, 0.7247191011235955)
(25, 0.7303370786516854)
(24, 0.7247191011235955)
(23, 0.7247191011235955)
(22, 0.7247191011235955)
(21, 0.7247191011235955)
(20, 0.7191011235955056)
(19, 0.7247191011235955)
(18, 0.7247191011235955)
(17, 0.7247191011235955)
(16, 0.7303370786516854)
(15, 0.7191011235955056)
(14, 0.7359550561797753)
(13, 0.7584269662921348)
(12, 0.7752808988764045)
(11, 0.7696629213483146)
(10, 0.7921348314606742)
(9, 0.7752808988764045)
(8, 0.7752808988764045)
(7, 0.7471910112359551)
(6, 0.7752808988764045)
(5, 0.7865168539325843)
(4, 0.8258426966292135)
(3, 0.8707865168539326)
(2, 0.8764044943820225)
(1, 1.0)


 ## Introduction to training and test sets
 
Our model has already seen and fit on the train data that we are using to produce an accuracy score.

![](https://cdn-images-1.medium.com/max/2000/1*-8_kogvwmL1H6ooN1A1tsQ.png)

Let's create a test that simulates fresh data that model might be predicting on when it is put into production.

In [123]:
# A:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine_df, y)  #order matters!!!
#when k is low it is simple to outliers 

Are our choice of hyper-parameters and features the same as they were when we were validating based on a training set alone?

In [126]:
for k in range(1,30)[::-1]:
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,y_train)
    print(k, knn.score(X_test,y_test))

(29, 0.64444444444444449)
(28, 0.66666666666666663)
(27, 0.64444444444444449)
(26, 0.64444444444444449)
(25, 0.64444444444444449)
(24, 0.64444444444444449)
(23, 0.62222222222222223)
(22, 0.64444444444444449)
(21, 0.62222222222222223)
(20, 0.66666666666666663)
(19, 0.64444444444444449)
(18, 0.64444444444444449)
(17, 0.62222222222222223)
(16, 0.64444444444444449)
(15, 0.59999999999999998)
(14, 0.66666666666666663)
(13, 0.59999999999999998)
(12, 0.62222222222222223)
(11, 0.66666666666666663)
(10, 0.71111111111111114)
(9, 0.66666666666666663)
(8, 0.71111111111111114)
(7, 0.66666666666666663)
(6, 0.64444444444444449)
(5, 0.66666666666666663)
(4, 0.66666666666666663)
(3, 0.68888888888888888)
(2, 0.64444444444444449)
(1, 0.71111111111111114)


<a id='resources'></a>

## Additional resources

---


- Scott Foreman-Roe's [breakdown](http://scott.fortmann-roe.com/docs/BiasVariance.html) (required) of the bias-variance tradeoff featuring a discussion of KNN is an excellent read
- A [detailed discussion](https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/) of KNN
- A long, applied example of KNN applied to [image classification](http://cs231n.github.io/classification/ )
- If academic breakdowns are your thing, be sure to visit [this](http://me.seekingqed.com/files/intro_KNN.pdf) resource
- Read the SKLearn [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) on implementing KNN
- Choosing the right [algorithm from SKLearn](http://scikit-learn.org/stable/tutorial/machine_learning_map/)
- A deeper dive into [Euclidian distance](http://www.econ.upf.edu/~michael/stanford/maeb4.pdf)
- Classifier comparsion from [SKLearn](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) (this is also in our [repository](https://github.com/ga-students/DSI-DC-2/blob/master/curriculum/Week-04/4.01%20Intro%20to%20Classification/classification-methods.py))